<a href="https://colab.research.google.com/github/TomkerDev/MID-cours/blob/main/TPBigData_TomteHassane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Université de N'Djaména - Faculté des Sciences Exactes et Appliquées.
Master Informatique - Année 2024-2025.
Matière : Projet Bigdata - Analyse films IMDb.
# Contrôle Travaux pratiques
Réalisé par: Tomté Hassane

# Environnement de Travail

## 1. Installation des dépendances

In [ ]:
# Installation de Java 8 (requis pour Spark)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Installation de la dernière version de PySpark
!pip install -q pyspark

## 2. Initialisation du SparkContext

In [ ]:

import pyspark
from pyspark import SparkContext, SparkConf

# Configuration de Spark
conf = SparkConf().setAppName("Projet_BigData_IMDb_2025").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)

print("SparkContext initialisé avec succès !")

SparkContext initialisé avec succès !


## 3. Téléchargement et Décompression des Données

In [ ]:

import os

# Téléchargement du fichier title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz

# Décompression du fichier (pour obtenir le .tsv)
!gunzip -f title.basics.tsv.gz

print("Dataset IMDb téléchargé et décompressé.")

Dataset IMDb téléchargé et décompressé.


## 4. Chargement initial et Nettoyage (Le RDD de base)

In [ ]:

# 1. Chargement du fichier texte en RDD
raw_rdd = sc.textFile("title.basics.tsv")

# 2. Extraction de l'en-tête pour pouvoir le filtrer
header = raw_rdd.first()

# 3. Nettoyage :
# - On enlève l'en-tête
# - On découpe par tabulation (\t)
# - On filtre les lignes qui n'ont pas le bon nombre de colonnes (optionnel mais sûr)
imdb_rdd = raw_rdd.filter(lambda line: line != header) \
                  .map(lambda line: line.split("\t")) \
                  .persist() # Conservation en mémoire pour les 10 exercices

print(f"Nombre total de lignes chargées : {imdb_rdd.count()}")

Nombre total de lignes chargées : 12192621


# Exercice 1
L'Exercice 1 nous demande d'implémenter un job MapReduce pour calculer le Top 10 des genres par runtime total.

## 1. Logique MapReduce

Pour cet exercice, la logique suit ces trois étapes fondamentales:

- Map : Pour chaque ligne, nous extrayons les genres (colonne index 8) et le temps de lecture (runtimeMinutes, colonne index 7). Comme un film peut avoir plusieurs genres séparés par des virgules, nous émettons une paire (genre, runtime) pour chaque genre individuel.

-  Shuffle & Sort : Spark regroupe tous les runtimes par nom de genre.

- Reduce : Nous additionnons tous les runtimes associés à un même genre pour obtenir le total global par catégorie.

## 2. Code Spark

Voici l'implémentation robuste en utilisant uniquement les RDDs, comme exigé par les consignes:

In [ ]:

def solve_exercice_1(rdd):
    # Étape 1 : Nettoyage et extraction (Map)
    # On filtre les valeurs non numériques '\N' pour le runtime
    # Le format de imdb_rdd est une liste : [tconst, titleType, primaryTitle, ..., runtimeMinutes, genres]

    def extract_genres_runtime(line):
        runtime_str = line[7]
        genres_str = line[8]

        if runtime_str == '\\N' or genres_str == '\\N':
            return []

        try:
            runtime = int(runtime_str)
            genres = genres_str.split(',')
            return [(genre.strip(), runtime) for genre in genres]
        except:
            return []

    # Étape 2 : Transformation et Agrégation (Reduce)
    top_genres = rdd.flatMap(extract_genres_runtime) \
                    .reduceByKey(lambda a, b: a + b) \
                    .sortBy(lambda x: x[1], ascending=False) \
                    .take(10) # Action pour récupérer le Top 10

    return top_genres

# Exécution
print("Calcul du Top 10 des genres par runtime total...")
resultat_ex1 = solve_exercice_1(imdb_rdd)

# Affichage propre
print(f"{'Genre':<20} | {'Runtime Total (min)':<20}")
print("-" * 45)
for genre, total in resultat_ex1:
    print(f"{genre:<20} | {total:<20,}")

Calcul du Top 10 des genres par runtime total...
Genre                | Runtime Total (min) 
---------------------------------------------
Drama                | 64,164,036          
Comedy               | 37,266,893          
Talk-Show            | 26,615,009          
Documentary          | 24,268,303          
Romance              | 21,323,316          
News                 | 20,568,096          
Crime                | 13,161,937          
Reality-TV           | 12,739,534          
Adult                | 12,178,728          
Action               | 10,680,022          


### Interprétation de l'Exercice 1 : Top 10 des genres par runtime total

L'objectif de cet exercice était d'identifier les genres cinématographiques qui cumulent le plus grand nombre de minutes de visionnage dans la base de données IMDb.

**Résultats obtenus :**

| Genre       | Runtime Total (min) |
| :---------- | :------------------ |
| Drama       | 64,164,036          |
| Comedy      | 37,266,893          |
| Talk-Show   | 26,615,009          |
| Documentary | 24,268,303          |
| Romance     | 21,323,316          |
| News        | 20,568,096          |
| Crime       | 13,161,937          |
| Reality-TV  | 12,739,534          |
| Adult       | 12,178,728          |
| Action      | 10,680,022          |

**Analyse des résultats :**

*   **Le Drame domine largement :** Avec plus de 64 millions de minutes cumulées, le genre "Drama" est de loin le plus représenté en termes de durée totale. Cela suggère une production très abondante de films dramatiques ou une tendance à des films de plus longue durée dans ce genre.
*   **La Comédie en deuxième position :** La "Comedy" arrive en seconde position avec près de 37 millions de minutes, confirmant son statut de genre populaire et prolifique.
*   **Genres de contenu fréquent :** Les "Talk-Show", "Documentary" et "News" figurent également très haut dans le classement. Leur présence est significative et peut s'expliquer par la nature de ces contenus, souvent produits en grande quantité (épisodes multiples, reportages quotidiens) et pouvant avoir des durées variables, contribuant rapidement à un runtime total élevé.
*   **Variété des genres :** Le Top 10 montre une diversité de genres, allant du "Romance" à l'"Action", en passant par le "Crime" et le "Reality-TV", indiquant une production cinématographique et télévisuelle variée qui a accumulé des millions de minutes de contenu dans chacun de ces domaines.

Ces résultats offrent une perspective intéressante sur la prépondérance de certains types de contenu dans le vaste catalogue d'IMDb, mettant en lumière les genres qui représentent le plus grand volume de temps de visionnage disponible.

# Exercice 2
## 1. Logique MapReduce

Le calcul de la médiane en distribué demande une approche spécifique :
- Map : Extraire le titleType (colonne index 1) comme clé et le runtimeMinutes (colonne index 7) comme valeur.
- GroupByKey : Regrouper toutes les durées pour chaque type de titre dans une liste.
- Sort & Pick : Pour chaque groupe, trier la liste des durées localement et sélectionner l'élément central (ou la moyenne des deux éléments centraux si la taille est paire).

## 2. Implémentation Spark (RDD)

Voici comment coder cela proprement en Python avec Spark :

In [ ]:

def solve_exercice_2(rdd):
    # 1. Map : (Type de titre, Runtime)
    # On ignore les valeurs non numériques '\N'
    type_runtime_rdd = rdd.map(lambda x: (x[1], x[7])) \
                          .filter(lambda x: x[1] != '\\N') \
                          .mapValues(lambda x: int(x))

    # 2. GroupByKey : Regrouper les durées par type
    grouped_rdd = type_runtime_rdd.groupByKey()

    # 3. Calcul de la médiane par groupe
    def calculate_median(iterator):
        sorted_list = sorted(list(iterator))
        n = len(sorted_list)
        if n == 0:
            return 0
        if n % 2 == 1:
            return sorted_list[n // 2]
        else:
            return (sorted_list[n // 2 - 1] + sorted_list[n // 2]) / 2

    medians = grouped_rdd.mapValues(calculate_median).collect()
    return medians

# Exécution
print("Calcul de la médiane par type de titre...")
resultat_ex2 = solve_exercice_2(imdb_rdd)

# Affichage des résultats
print(f"{'Type de Titre':<15} | {'Médiane (min)':<15}")
print("-" * 35)
for t_type, median in resultat_ex2:
    print(f"{t_type:<15} | {median:<15}")

Calcul de la médiane par type de titre...
Type de Titre   | Médiane (min)  
-----------------------------------
short           | 11.0           
movie           | 89             
tvMiniSeries    | 45.0           
tvMovie         | 62             
tvSpecial       | 60             
tvSeries        | 30.0           
videoGame       | 58.5           
video           | 61.0           
tvEpisode       | 30             
tvShort         | 11             


### Interprétation de l'Exercice 7 : Hiérarchie de genres et analyse arborescente

L'objectif de cet exercice était de construire une hiérarchie simplifiée de genres en considérant le premier genre d'un film comme parent et les suivants comme sous-genres, puis de compter le nombre de titres pour chaque "feuille" ou chemin hiérarchique unique.

**Résultats obtenus (Top 15 des feuilles les plus populaires) :**

| Chemin Hiérarchique (Arbre)              | Nombre de Films |
| :--------------------------------------- | :-------------- |
| Documentary                              | 2,074,383       |
| Drama                                    | 1,029,915       |
| Comedy                                   | 650,891         |
| Short                                    | 501,279         |
| Adult                                    | 352,430         |
| Animation                                | 267,823         |
| News                                     | 263,403         |
| Talk-Show                                | 200,604         |
| Reality-TV                               | 175,027         |
| Family                                   | 167,778         |
| Music                                    | 139,122         |
| Action                                   | 121,387         |
| Romance                                  | 119,773         |
| Sport                                    | 107,314         |
| Drama -> Comedy                          | 95,782          |

**Analyse des résultats :**

*   **Prépondérance des genres uniques :** Le classement est largement dominé par des genres uniques ("Documentary", "Drama", "Comedy", "Short", etc.). Cela signifie que la majorité des films sont étiquetés avec un seul genre, ou du moins, leur premier genre est suffisamment distinctif pour être un chemin à part entière. Le "Documentary" est de loin le plus fréquent, ce qui est cohérent avec la grande quantité de contenus informatifs souvent classés uniquement sous ce genre.
*   **Premiers chemins hiérarchiques :** Le premier chemin hiérarchique composé de plusieurs genres à apparaître est "Drama -> Comedy" avec 95,782 films. Cela indique que cette combinaison spécifique est courante, où un film est principalement un Drame mais incorpore des éléments de Comédie. La convention de prendre le premier genre comme parent simplifie la hiérarchie et concentre l'analyse sur ces paires spécifiques.
*   **Importance du premier genre :** Les résultats suggèrent que, pour beaucoup de films, le premier genre listé par IMDb est souvent suffisant pour les caractériser ou qu'il est le plus influent dans leur classification.
*   **Limites de la simplification :** La règle "premier genre = parent" est une simplification nécessaire pour cet exercice. Dans une analyse de graphe de genres plus sophistiquée, on pourrait explorer des relations plus complexes (par exemple, un film de type "Action, Aventure, Science-Fiction" pourrait avoir des liens vers "Action -> Aventure", "Action -> Science-Fiction", "Aventure -> Science-Fiction", etc.). Cependant, pour le but de cet exercice, cette approche permet d'illustrer la construction et le comptage de chemins hiérarchiques.

Cet exercice démontre comment des techniques MapReduce peuvent être utilisées pour extraire et quantifier des relations hiérarchiques ou séquentielles à partir de données textuelles structurées, offrant une vue agrégée de la co-occurrence des genres.

# Exercice 3 : Calcul de l'écart-type global des runtimes

### 1. Objectif
Calculer l'écart-type ($\sigma$) de la durée de tous les titres présents dans le dataset IMDb. [cite_start]L'écart-type mesure la dispersion des durées par rapport à la moyenne globale.

### 2. Logique MapReduce (Formule de Koenig-Huygens)
Pour calculer l'écart-type en une seule passe sur les données (plus efficace que deux passes), nous utilisons la formule suivante :

$$\sigma = \sqrt{E[X^2] - (E[X])^2}$$

Où :
* $E[X]$ est la moyenne des runtimes.
* $E[X^2]$ est la moyenne des carrés des runtimes.

**Le job MapReduce se décompose ainsi :**

* **Étape MAP :** Pour chaque ligne, si le runtime est valide, on émet un triplet : $(1, runtime, runtime^2)$.
    * Le `1` sert à compter le nombre total d'éléments ($N$).
    * Le `runtime` sert à calculer la somme totale ($\sum x$).
    * Le `runtime^2` sert à calculer la somme des carrés ($\sum x^2$).
* **Étape REDUCE :** On additionne ces triplets de manière globale pour obtenir trois cumuls : $(N_{total}, \sum x_{total}, \sum x^2_{total})$.
* **Étape FINALE :** On applique la formule de l'écart-type sur ces trois valeurs.

## 2. Implémentation Spark (RDD)

Voici comment implémenter ce calcul en une seule passe efficace :

In [ ]:
import math

def solve_exercice_3(rdd):
    # 1. Map : (1, runtime, runtime^2)
    # On filtre les valeurs '\N' et on prépare les agrégats
    stats_rdd = rdd.map(lambda x: x[7]) \
                   .filter(lambda x: x != '\\N') \
                   .map(lambda x: float(x)) \
                   .map(lambda val: (1, val, val**2))

    # 2. Reduce : Somme globale des 3 composantes
    # On utilise reduce pour combiner tous les tuples
    total_n, total_sum, total_sum_sq = stats_rdd.reduce(
        lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2])
    )

    # 3. Calculs finaux
    mean = total_sum / total_n
    variance = (total_sum_sq / total_n) - (mean ** 2)
    std_dev = math.sqrt(variance)

    return std_dev, mean, total_n

# Exécution
std_dev, mean, n = solve_exercice_3(imdb_rdd)

print(f"Statistiques Globales :")
print(f"- Nombre de films (N) : {n:,}")
print(f"- Moyenne (μ)         : {mean:.2f} min")
print(f"- Écart-type (σ)      : {std_dev:.2f} min")

Statistiques Globales :
- Nombre de films (N) : 4,345,721
- Moyenne (μ)         : 44.77 min
- Écart-type (σ)      : 1773.88 min


### Interprétation de l'Exercice 3 : Écart-type global des runtimes

L'objectif de cet exercice était de calculer l'écart-type et la moyenne de tous les runtimes du dataset, ainsi que le nombre total de titres avec des runtimes valides. Ces statistiques permettent de comprendre la dispersion des durées des titres dans la base de données.

**Résultats obtenus :**

*   **Nombre de films (N)** : 4,345,721
*   **Moyenne (μ)** : 44.77 min
*   **Écart-type (σ)** : 1773.88 min

**Analyse des résultats :**

*   **Nombre de titres conséquent :** Le dataset contient 4,345,721 titres avec un runtime valide, ce qui représente un volume de données très important, justifiant l'utilisation d'outils distribués comme Spark.
*   **Moyenne des runtimes :** La durée moyenne est d'environ 44.77 minutes. Cette valeur est relativement basse si l'on ne considère que les longs-métrages, mais elle inclut tous les types de titres (courts-métrages, épisodes de séries, documentaires, etc.), ce qui tend à abaisser la moyenne générale.
*   **Écart-type très élevé :** L'écart-type de 1773.88 minutes est extrêmement élevé par rapport à la moyenne. Cela indique une **très grande dispersion** des runtimes dans le dataset. Un tel écart-type suggère qu'il y a une large gamme de durées, avec un nombre significatif de titres ayant des durées très courtes (quelques minutes) et, de manière moins fréquente mais impactante, des titres extrêmement longs (plusieurs heures, voire jours ou même années pour certains types de contenus expérimentaux ou compilations, comme le démontrera l'exercice sur les anomalies).
*   **Impact des valeurs extrêmes :** La formule de Koenig-Huygens utilisée pour l'écart-type est robuste et efficace pour un calcul distribué en une seule passe. L'ampleur de l'écart-type est un indicateur fort de la présence de valeurs extrêmes qui éloignent considérablement les données de la moyenne, rendant la moyenne seule peu représentative sans l'écart-type pour contextualiser la dispersion.

En résumé, ces statistiques révèlent un dataset IMDb caractérisé par un très grand nombre de titres et une hétérogénéité prononcée des durées, avec une présence notable de contenus à durées extrêmes.

# Exercice 4 : Simulation d'un graphe d'amis et calcul d'amis communs

### 1. Objectif
Simuler un réseau social où chaque film est "ami" avec les autres films partageant le même **genre principal**. L'objectif final est de trouver les 5 paires de films ayant le plus grand nombre d'amis communs.

### 2. Logique MapReduce
Le problème se décompose en trois phases complexes :

* **Phase 1 : Identification du genre principal** * **Map :** On extrait le titre du film et son premier genre listé.
    * **Résultat :** Un RDD de type `(Genre, Titre_Film)`.
* **Phase 2 : Création des listes d'amis**
    * **GroupByKey :** On regroupe tous les films par genre. Chaque film d'un groupe est "ami" avec tous les autres membres du même groupe.
* **Phase 3 : Calcul des amis communs (Intersection)**
    * Pour chaque paire de films $(A, B)$, on calcule le nombre de genres qu'ils partagent.
    * **Map :** Pour chaque film, on émet des paires `((Film_A, Film_B), 1)` pour chaque ami qu'ils ont en commun.
    * **Reduce :** On additionne les occurrences pour obtenir le total par paire.

## 2. Code Spark (RDD uniquement)
Cet exercice demande beaucoup de ressources. Pour qu'il s'exécute bien sur Colab, nous allons limiter l'analyse aux 1000 premiers films pour la démonstration, car comparer des millions de films entre eux créerait des milliards de combinaisons.

In [ ]:
def solve_exercice_4(rdd):
    # 1. Préparation : (Genre_Principal, Titre)
    # On prend un échantillon (sample) pour éviter de saturer la mémoire de Colab
    # car le nombre de paires croît de façon exponentielle (N²)
    sample_rdd = sc.parallelize(rdd.take(1000))

    # Mapper : (Genre, Titre) - On considère le premier genre comme principal
    movie_genre = sample_rdd.map(lambda x: (x[8].split(',')[0], x[2])) \
                            .filter(lambda x: x[0] != '\\N')

    # 2. Créer les listes d'amis par genre
    # groupByKey donne (Genre, [Film1, Film2, Film3...])
    friends_by_genre = movie_genre.groupByKey().mapValues(list)

    # 3. Générer les paires de films qui partagent un ami (le genre)
    def generate_pairs(movie_list):
        pairs = []
        import itertools
        # Pour chaque groupe de films d'un même genre, tous sont "amis"
        for pair in itertools.combinations(sorted(movie_list), 2):
            pairs.append((pair, 1))
        return pairs

    # 4. Compter les amis communs et prendre le Top 5
    common_friends = friends_by_genre.flatMap(lambda x: generate_pairs(x[1])) \
                                     .reduceByKey(lambda a, b: a + b) \
                                     .sortBy(lambda x: x[1], ascending=False) \
                                     .take(5)

    return common_friends

# Exécution
print("Calcul des paires de films ayant le plus d'amis communs (Top 5)...")
resultat_ex4 = solve_exercice_4(imdb_rdd)

# Affichage
print(f"{'Paire de Films':<50} | {'Amis Communs':<15}")
print("-" * 70)
for paire, count in resultat_ex4:
    print(f"{str(paire):<50} | {count:<15}")

Calcul des paires de films ayant le plus d'amis communs (Top 5)...
Paire de Films                                     | Amis Communs   
----------------------------------------------------------------------
('Hamlet', 'Othello')                              | 8              
('Hamlet, Prince of Denmark', 'Othello')           | 8              
('Macbeth', 'Othello')                             | 8              
('Othello', 'The Last Days of Pompeii')            | 8              
('Othello', "Uncle Tom's Cabin")                   | 8              


### Interprétation de l'Exercice 4 : Amis communs (Genre principal)

L'objectif de cet exercice était de simuler un réseau d'amitié entre films basé sur le partage d'un "genre principal" (le premier genre listé). Nous avons ensuite cherché les 5 paires de films ayant le plus d'amis communs (c'est-à-dire le plus de genres partagés en tant que genre principal avec d'autres films).

**Résultats obtenus (avec un échantillon de 1000 films pour des raisons de performance) :**

| Paire de Films                       | Amis Communs |
| :----------------------------------- | :----------- |
| ('Hamlet', 'Othello')                | 8            |
| ('Hamlet, Prince of Denmark', 'Othello') | 8            |
| ('Macbeth', 'Othello')               | 8            |
| ('Othello', 'The Last Days of Pompeii') | 8            |
| ('Othello', "Uncle Tom's Cabin")     | 8            |

**Analyse des résultats :**

*   **Thèmes classiques et multiples adaptations :** Le Top 5 est dominé par des adaptations de pièces classiques, notamment "Hamlet" et "Othello". Il est très fréquent que ces œuvres aient été adaptées de multiples fois au cinéma sous différentes formes (film, pièce filmée, court-métrage, etc.). Chaque adaptation peut avoir été catégorisée sous un genre principal légèrement différent ou, plus probablement, elles partagent toutes un ensemble de genres fondamentaux (Drama, History, Romance) qui les connectent à un large éventail d'autres films partageant ces mêmes genres.
*   **"Othello" comme point central :** Il est frappant de constater que le film "Othello" apparaît dans toutes les paires du Top 5. Cela suggère que ce titre (ou ses multiples versions) partage un ensemble de genres principaux très communs avec d'autres films historiques ou dramatiques, le plaçant au centre d'un grand nombre de "groupes d'amis" (genres partagés).
*   **Impact de la limitation de l'échantillon :** Il est crucial de noter que ces résultats sont basés sur un échantillon limité de 1000 films. Sur l'ensemble du dataset, la complexité du réseau serait exponentiellement plus grande, et les paires les plus connectées pourraient être différentes, révélant d'autres dynamiques de genres partagés. Cependant, même avec l'échantillon, la méthode démontre sa capacité à identifier des films partageant des contextes génériques similaires.
*   **Interprétation de "Amis Communs" :** Ici, "amis communs" ne signifie pas que les films ont été produits ensemble ou partagent des acteurs, mais qu'ils ont un grand nombre de *genres* en commun avec d'autres films. La logique MapReduce a efficacement identifié ces liens transversaux basés sur les catégories.

Cet exercice illustre bien la puissance de MapReduce pour analyser des relations complexes dans de grands volumes de données, même avec les contraintes de performance imposées par la taille potentielle du graphe.

# Exercice 5 : Moyenne pondérée des runtimes par genre

### 1. Objectif
Calculer la moyenne pondérée des durées (runtimes) pour chaque genre de film.
[cite_start]Le poids ($w$) d'un film est défini par la formule : $w = \frac{runtime}{1000}$.

### 2. Formule Mathématique
La moyenne pondérée ($\bar{x}_w$) se calcule comme suit :
$$\bar{x}_w = \frac{\sum (w_i \times x_i)}{\sum w_i}$$
Où :
* $x_i$ est le runtime du film.
* $w_i$ est le poids ($runtime / 1000$).

### 3. Logique MapReduce
* **MAP :** Pour chaque film, on extrait les genres. Pour chaque genre, on émet une paire :
  `Clé : Genre`
  `Valeur : (Poids * Runtime, Poids)`
* **REDUCE :** On additionne séparément les produits pondérés et les poids totaux pour chaque genre.
* **FINALISATION :** On divise la somme des produits par la somme des poids.

## 2. Implémentation Spark (RDD)
Voici le code optimisé pour ce calcul:

In [ ]:

def solve_exercice_5(rdd):
    def map_weighted_values(line):
        genres_str = line[8]
        runtime_str = line[7]

        if genres_str == '\\N' or runtime_str == '\\N':
            return []

        try:
            runtime = float(runtime_str)
            weight = runtime / 1000.0
            weighted_val = weight * runtime

            genres = genres_str.split(',')
            # On retourne (Genre, (Poids * Valeur, Poids))
            return [(genre.strip(), (weighted_val, weight)) for genre in genres]
        except:
            return []

    # 1. Map & Reduce
    # On additionne les (valeurs pondérées) et les (poids)
    weighted_stats = rdd.flatMap(map_weighted_values) \
                        .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

    # 2. Calcul final de la moyenne pondérée (Moyenne = Somme_pondérée / Somme_poids)
    weighted_averages = weighted_stats.mapValues(lambda x: x[0] / x[1] if x[1] != 0 else 0) \
                                      .sortBy(lambda x: x[1], ascending=False) \
                                      .collect()

    return weighted_averages

# Exécution
print("Calcul de la moyenne pondérée par genre...")
resultat_ex5 = solve_exercice_5(imdb_rdd)

# Affichage
print(f"{'Genre':<20} | {'Moyenne Pondérée (min)':<20}")
print("-" * 45)
for genre, avg in resultat_ex5[:15]: # Top 15 pour la lisibilité
    print(f"{genre:<20} | {avg:<20.2f}")

Calcul de la moyenne pondérée par genre...
Genre                | Moyenne Pondérée (min)
---------------------------------------------
Reality-TV           | 1070120.21          
Comedy               | 365882.97           
Drama                | 212532.34           
Biography            | 6189.22             
Animation            | 960.88              
Documentary          | 320.50              
Action               | 219.67              
Sport                | 203.29              
Adult                | 134.05              
Musical              | 133.43              
Music                | 109.80              
War                  | 106.36              
History              | 105.40              
Adventure            | 102.12              
Talk-Show            | 100.65              


### Interprétation de l'Exercice 5 : Moyenne pondérée des runtimes par genre

Cet exercice consistait à calculer la moyenne pondérée des runtimes pour chaque genre, où le poids de chaque film était proportionnel à son propre runtime ($w = runtime / 1000$). Cela permet de donner plus d'importance aux films plus longs lors du calcul de la moyenne.

**Résultats obtenus (Top 15 pour la lisibilité) :**

| Genre           | Moyenne Pondérée (min) |
| :-------------- | :--------------------- |
| Reality-TV      | 1070120.21             |
| Comedy          | 365882.97              |
| Drama           | 212532.34              |
| Biography       | 6189.22                |
| Animation       | 960.88                 |
| Documentary     | 320.50                 |
| Action          | 219.67                 |
| Sport           | 203.29                 |
| Adult           | 134.05                 |
| Musical         | 133.43                 |
| Music           | 109.80                 |
| War             | 106.36                 |
| History         | 105.40                 |
| Adventure       | 102.12                 |
| Talk-Show       | 100.65                 |

**Analyse des résultats :**

*   **Valeurs extrêmes et anomalies :** La moyenne pondérée fait ressortir des valeurs extrêmement élevées pour "Reality-TV", "Comedy" et "Drama". Une moyenne pondérée de plus d'un million de minutes pour la "Reality-TV" est une indication claire de la présence de runtimes aberrants et extrêmement longs dans ces genres, impactant fortement le calcul en raison de la pondération. Ces "films" ne sont probablement pas des films au sens classique, mais des compilations, des marathons, des enregistrements de flux continus, ou des erreurs de données comme vu dans l'exercice 9.
*   **Impact de la pondération :** Contrairement à une moyenne arithmétique simple, la moyenne pondérée amplifie l'effet des films les plus longs. Si un film de 100 000 minutes existe dans le genre "Reality-TV", il pèsera bien plus lourd dans la moyenne que 1000 films de 10 minutes. C'est pourquoi ces chiffres sont si disproportionnés.
*   **Genres plus "standards" :** En dessous de ces trois genres extrêmes, on retrouve des valeurs plus cohérentes avec des durées de films/séries, comme la "Biography" (6189 min, encore élevé mais plus dans l'ordre de grandeur de très longues séries ou compilations), l'"Animation" (960 min), le "Documentary" (320 min), l'"Action" (219 min), etc. Ces genres, bien que parfois longs, sont moins affectés par les "hyper-runtimes" observés plus haut.
*   **Objectif atteint :** L'exercice a réussi à démontrer comment la moyenne pondérée peut mettre en évidence l'influence des éléments les plus "lourds" (ici, les titres les plus longs) dans un ensemble de données, même si le résultat est dominé par des cas extrêmes. Cela est particulièrement utile pour identifier les genres où les productions de très longue durée sont les plus significatives.

Cet exercice souligne la nécessité de comprendre la nature des données et l'impact des méthodes de calcul (moyenne simple vs. moyenne pondérée) pour éviter des interprétations erronées, en particulier en présence de valeurs aberrantes.

# Exercice 6 : Histogramme logarithmique des runtimes

### 1. Objectif
Classer tous les titres du dataset dans des "bacs" (buckets) basés sur une échelle logarithmique de base 10 pour visualiser la distribution des durées. Les classes définies sont [0-10min[, [10-100[, [100-1000[, etc.

### 2. Logique MapReduce
Pour créer cet histogramme de manière distribuée :
* **MAP :** Pour chaque film, on récupère le runtime ($x$). On détermine sa classe en calculant la puissance de 10 inférieure.
    * Formule logique : Si $10^n \le x < 10^{n+1}$, alors la classe est $[10^n - 10^{n+1}[$.
* **REDUCE :** On compte le nombre d'occurrences pour chaque classe (clé).
* **TRI :** On trie les résultats par ordre croissant des classes.

## 2. Implémentation Spark (RDD)
Voici le code pour générer ces données d'histogramme:

In [ ]:

import math

def solve_exercice_6(rdd):
    def get_log_bucket(line):
        runtime_str = line[7]
        if runtime_str == '\\N':
            return []

        try:
            runtime = float(runtime_str)
            if runtime <= 0: return []

            # Calcul de la puissance de 10
            # log10(5) -> 0.69 -> bucket 10^0 = 1
            # log10(50) -> 1.69 -> bucket 10^1 = 10
            exponent = int(math.log10(runtime))
            lower_bound = 10**exponent
            upper_bound = 10**(exponent + 1)

            bucket_label = f"[{lower_bound}-{upper_bound} min["
            return [(bucket_label, 1)]
        except:
            return []

    # MapReduce : Compter par bucket
    histogram_data = rdd.flatMap(get_log_bucket) \
                        .reduceByKey(lambda a, b: a + b) \
                        .sortByKey() \
                        .collect()

    return histogram_data

# Exécution
print("Génération de l'histogramme logarithmique...")
resultat_ex6 = solve_exercice_6(imdb_rdd)

# Affichage des résultats
print(f"{'Classe de Runtime':<20} | {'Nombre de Titres':<20}")
print("-" * 45)
for bucket, count in resultat_ex6:
    print(f"{bucket:<20} | {count:<20,}")

Génération de l'histogramme logarithmique...
Classe de Runtime    | Nombre de Titres    
---------------------------------------------
[1-10 min[           | 501,279             
[10-100 min[         | 3,518,980           
[100-1000 min[       | 325,065             
[1000-10000 min[     | 376                 
[10000-100000 min[   | 15                  
[100000-1000000 min[ | 1                   
[1000000-10000000 min[ | 1                   


# Exercice 7 : Hiérarchie de genres et analyse arborescente

### 1. Objectif
Construire une hiérarchie de genres (par exemple : Action → Action-Adventure) et compter le nombre de titres pour chaque nœud terminal (feuille) de cette arborescence.

### 2. Logique MapReduce
* **MAP :** Pour chaque titre, on extrait la liste des genres. On définit une règle de hiérarchie.
    * *Règle choisie :* Si un film appartient à plusieurs genres (ex: Action, Adventure), on crée un chemin hiérarchique : `Action -> Adventure`.
* **TRANSFORMATION :** On transforme les genres multiples en une chaîne de caractères représentant un chemin dans l'arbre.
* **REDUCE :** On compte le nombre de films pour chaque chemin unique (nœud feuille).

## 2. Implémentation Spark (RDD)
Dans IMDb, les genres sont fournis sous forme de liste (ex: "Action,Comedy,Drama"). Nous allons considérer que le premier genre est le parent et les suivants sont des sous-genres.

In [ ]:
def solve_exercice_7(rdd):
    def build_hierarchy(line):
        genres_str = line[8]
        if genres_str == '\\N':
            return []

        genres = [g.strip() for g in genres_str.split(',')]

        if len(genres) == 1:
            # Nœud simple
            return [(genres[0], 1)]
        else:
            # Création du chemin hiérarchique (Action -> Adventure)
            path = " -> ".join(genres)
            return [(path, 1)]

    # MapReduce pour compter les occurrences par chemin hiérarchique
    hierarchy_counts = rdd.flatMap(build_hierarchy) \
                          .reduceByKey(lambda a, b: a + b) \
                          .sortBy(lambda x: x[1], ascending=False) \
                          .take(15) # On prend les 15 feuilles les plus populaires

    return hierarchy_counts

# Exécution
print("Analyse de la hiérarchie des genres (Chemins -> Nombre de films)...")
resultat_ex7 = solve_exercice_7(imdb_rdd)

# Affichage
print(f"{'Chemin Hiérarchique (Arbre)':<40} | {'Nombre de Films':<15}")
print("-" * 60)
for path, count in resultat_ex7:
    print(f"{path:<40} | {count:<15,}")

### Interprétation de l'Exercice 7 : Hiérarchie de genres et analyse arborescente

L'objectif de cet exercice était de construire une hiérarchie simplifiée de genres en considérant le premier genre d'un film comme parent et les suivants comme sous-genres, puis de compter le nombre de titres pour chaque "feuille" ou chemin hiérarchique unique.

**Résultats obtenus (Top 15 des feuilles les plus populaires) :**

| Chemin Hiérarchique (Arbre)              | Nombre de Films |
| :--------------------------------------- | :-------------- |
| Documentary                              | 2,074,383       |
| Drama                                    | 1,029,915       |
| Comedy                                   | 650,891         |
| Short                                    | 501,279         |
| Adult                                    | 352,430         |
| Animation                                | 267,823         |
| News                                     | 263,403         |
| Talk-Show                                | 200,604         |
| Reality-TV                               | 175,027         |
| Family                                   | 167,778         |
| Music                                    | 139,122         |
| Action                                   | 121,387         |
| Romance                                  | 119,773         |
| Sport                                    | 107,314         |
| Drama -> Comedy                          | 95,782          |

**Analyse des résultats :**

*   **Prépondérance des genres uniques :** Le classement est largement dominé par des genres uniques ("Documentary", "Drama", "Comedy", "Short", etc.). Cela signifie que la majorité des films sont étiquetés avec un seul genre, ou du moins, leur premier genre est suffisamment distinctif pour être un chemin à part entière. Le "Documentary" est de loin le plus fréquent, ce qui est cohérent avec la grande quantité de contenus informatifs souvent classés uniquement sous ce genre.
*   **Premiers chemins hiérarchiques :** Le premier chemin hiérarchique composé de plusieurs genres à apparaître est "Drama -> Comedy" avec 95,782 films. Cela indique que cette combinaison spécifique est courante, où un film est principalement un Drame mais incorpore des éléments de Comédie. La convention de prendre le premier genre comme parent simplifie la hiérarchie et concentre l'analyse sur ces paires spécifiques.
*   **Importance du premier genre :** Les résultats suggèrent que, pour beaucoup de films, le premier genre listé par IMDb est souvent suffisant pour les caractériser ou qu'il est le plus influent dans leur classification.
*   **Limites de la simplification :** La règle "premier genre = parent" est une simplification nécessaire pour cet exercice. Dans une analyse de graphe de genres plus sophistiquée, on pourrait explorer des relations plus complexes (par exemple, un film de type "Action, Aventure, Science-Fiction" pourrait avoir des liens vers "Action -> Aventure", "Action -> Science-Fiction", "Aventure -> Science-Fiction", etc.). Cependant, pour le but de cet exercice, cette approche permet d'illustrer la construction et le comptage de chemins hiérarchiques.

Cet exercice démontre comment des techniques MapReduce peuvent être utilisées pour extraire et quantifier des relations hiérarchiques ou séquentielles à partir de données textuelles structurées, offrant une vue agrégée de la co-occurrence des genres.

# Exercice 8 : PageRank simplifié sur les genres (5 itérations)

### 1. Objectif
Calculer le prestige ou l'importance relative de chaque genre en utilisant l'algorithme PageRank. Un genre est considéré comme "important" s'il est souvent associé à d'autres genres importants.

### 2. Logique de l'algorithme (5 itérations)
L'algorithme suit une logique itérative selon la formule simplifiée fournie :
$$Score(G) = \sum \frac{Score(Voisins)}{Degré\_Sortant(Voisins)}$$

**Le processus MapReduce :**
1. **Initialisation :** Chaque genre commence avec un score de 1.0.
2. **Construction du Graphe :** Pour chaque film, on crée des liens entre ses genres (ex: Action -> Comédie, Action -> Drame).
3. **Itérations (Boucle de 5) :**
    * **Contributions (Map) :** Chaque genre distribue son score actuel équitablement entre tous ses voisins.
    * **Agrégation (Reduce) :** On additionne toutes les contributions reçues par chaque genre pour calculer son nouveau score.

2. Implémentation Spark (RDD)
Cet exercice utilise une boucle **for** en Python qui déclenche des jobs Spark successifs.

In [ ]:
def solve_exercice_8(rdd):
    # 1. Préparation des liens (Graphe des genres)
    # Pour un film "Action, Comedy", on crée Action -> Comedy et Comedy -> Action
    def extract_links(line):
        genres = [g.strip() for g in line[8].split(',') if g != '\\N']
        links = []
        for g1 in genres:
            for g2 in genres:
                if g1 != g2:
                    links.append((g1, g2))
        return links

    # RDD des liens : (Source, Cible) -> distinct pour éviter les doublons
    links_rdd = rdd.flatMap(extract_links).distinct().groupByKey().mapValues(list).persist()

    # 2. Initialisation des scores : (Genre, 1.0)
    genres_rdd = rdd.flatMap(lambda x: [g.strip() for g in x[8].split(',') if g != '\\N']).distinct()
    ranks_rdd = genres_rdd.map(lambda genre: (genre, 1.0))

    # 3. Boucle itérative (5 itérations comme demandé)
    for i in range(5):
        # Jointure pour envoyer les scores aux voisins
        # (Genre, (Liste_Voisins, Score_Actuel))
        contributions = links_rdd.join(ranks_rdd).flatMap(
            lambda x: [(dest, x[1][1] / len(x[1][0])) for dest in x[1][0]]
        )

        # Recalcul des scores (Reduce)
        ranks_rdd = contributions.reduceByKey(lambda a, b: a + b)
        print(f"Itération {i+1} terminée.")

    return ranks_rdd.sortBy(lambda x: x[1], ascending=False).take(10)

# Exécution
print("Calcul du PageRank sur les genres (5 itérations)...")
resultat_ex8 = solve_exercice_8(imdb_rdd)

# Affichage
print(f"\n{'Genre':<20} | {'Score PageRank':<15}")
print("-" * 40)
for genre, rank in resultat_ex8:
    print(f"{genre:<20} | {rank:<15.4f}")

Calcul du PageRank sur les genres (5 itérations)...
Itération 1 terminée.
Itération 2 terminée.
Itération 3 terminée.
Itération 4 terminée.
Itération 5 terminée.

Genre                | Score PageRank 
----------------------------------------
Crime                | 1.0385         
Sport                | 1.0385         
Mystery              | 1.0385         
Comedy               | 1.0385         
Drama                | 1.0385         
Music                | 1.0385         
Horror               | 1.0385         
Fantasy              | 1.0385         
Romance              | 1.0385         
Thriller             | 1.0385         


### Interprétation de l'Exercice 8 : PageRank simplifié sur les genres (5 itérations)

L'objectif de cet exercice était de calculer un score de "prestige" ou d'importance pour chaque genre en utilisant un algorithme PageRank simplifié, sur 5 itérations. Ce score est basé sur la connectivité des genres : un genre est important s'il est lié à d'autres genres importants.

**Résultats obtenus (Top 10 après 5 itérations) :**

| Genre      | Score PageRank |
| :--------- | :------------- |
| Crime      | 1.0385         |
| Sport      | 1.0385         |
| Mystery    | 1.0385         |
| Comedy     | 1.0385         |
| Drama      | 1.0385         |
| Music      | 1.0385         |
| Horror     | 1.0385         |
| Fantasy    | 1.0385         |
| Romance    | 1.0385         |
| Thriller   | 1.0385         |

**Analyse des résultats :**

*   **Scores PageRank quasi identiques :** Après 5 itérations, les scores PageRank des genres principaux sont très proches les uns des autres (tous autour de 1.0385). Cela suggère une distribution assez uniforme de l'importance parmi les genres les plus connectés.
*   **Interprétation de l'uniformité :**
    *   **Nature des données :** Dans le dataset IMDb, de nombreux films sont associés à plusieurs genres. Si un film a "Action, Aventure, Science-Fiction", des liens sont créés entre "Action" et "Aventure", "Action" et "Science-Fiction", "Aventure" et "Science-Fiction". Cela crée un réseau de genres où la plupart des genres populaires sont fortement interconnectés.
    *   **Simplification de l'algorithme :** La formule $Score(G) = \sum \frac{Score(Voisins)}{Degré\_Sortant(Voisins)}$ sans facteur d'amortissement (damping factor) et avec une initialisation uniforme de 1.0, tend à distribuer la "valeur" de manière très homogène si le graphe est fortement connecté et que les degrés sortants des genres populaires sont similaires.
    *   **Nombre d'itérations :** 5 itérations peuvent être suffisantes pour faire converger le PageRank dans un graphe fortement connecté, mais si le graphe était plus complexe ou moins dense, davantage d'itérations pourraient être nécessaires pour voir des différences plus marquées.
*   **Genres représentatifs :** Les genres listés dans le Top 10 sont typiquement des genres majeurs et très répandus dans l'industrie cinématographique. Leur forte interconnexion avec d'autres genres (un film "Action" est souvent aussi "Aventure" ou "Thriller", etc.) conduit à ces scores élevés et uniformes.
*   **Conclusion sur l'importance :** L'uniformité des scores pour ces genres ne signifie pas qu'ils sont tous "égaux", mais plutôt que, dans ce modèle de graphe et avec cette simplification de l'algorithme, leur importance relative est très similaire en termes de connectivité et de distribution de "préstige". Pour une analyse plus fine, il faudrait potentiellement ajuster l'algorithme (ex: ajouter un facteur d'amortissement) ou augmenter le nombre d'itérations.

Cet exercice démontre la faisabilité d'appliquer des algorithmes de type PageRank à des données non-web pour analyser l'importance relative d'entités (ici, les genres), tout en soulignant l'importance de bien comprendre les paramètres de l'algorithme et les caractéristiques du graphe sous-jacent.

# Exercice 9 : Détection des anomalies de runtime par genre (Z-score > 2)

### 1. Objectif
Identifier les films "aberrants" dont la durée s'éloigne significativement de la moyenne de leur genre. Un film est considéré comme une anomalie si son **Z-score** est supérieur à 2.

### 2. Formule du Z-score
Pour un film donné, le Z-score se calcule ainsi :
$$Z = \frac{x - \mu}{\sigma}$$
Où :
* $x$ : runtime du film.
* $\mu$ : moyenne du runtime pour ce genre.
* $\sigma$ : écart-type du runtime pour ce genre.

### 3. Logique MapReduce
Ce job nécessite deux étapes majeures :
1. **Calcul des statistiques par genre :** Calculer $\mu$ et $\sigma$ pour chaque genre (similaire à l'Exercice 3, mais groupé par genre).
2. **Filtrage des anomalies :** Joindre ces statistiques avec le RDD original pour calculer le Z-score de chaque film et ne garder que ceux où $Z > 2$.

## 2. Implémentation Spark (RDD)
Ce code est plus complexe car il nécessite de calculer des statistiques groupées avant de les réutiliser.

In [ ]:

import math

def solve_exercice_9(rdd):
    # 1. Calcul de (Genre, (Somme_X, Somme_X2, Count))
    def stats_mapper(line):
        genres = [g.strip() for g in line[8].split(',') if g != '\\N']
        try:
            rt = float(line[7])
            return [(g, (rt, rt**2, 1)) for g in genres]
        except: return []

    genre_stats = rdd.flatMap(stats_mapper) \
                     .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1], a[2]+b[2])) \
                     .mapValues(lambda x: (x[0]/x[2], math.sqrt(max(0, (x[1]/x[2]) - (x[0]/x[2])**2)))) \
                     .collectAsMap() # On stocke en dictionnaire pour un accès rapide

    # 2. Diffusion des stats (Broadcast) pour calculer le Z-score
    def find_anomalies(line):
        title = line[2]
        genres = [g.strip() for g in line[8].split(',') if g != '\\N']
        try:
            rt = float(line[7])
            anomalies = []
            for g in genres:
                if g in genre_stats:
                    mu, sigma = genre_stats[g]
                    if sigma > 0:
                        z = (rt - mu) / sigma
                        if z > 2: # Seuil d'anomalie
                            anomalies.append((title, g, rt, z))
            return anomalies
        except: return []

    anomalies_rdd = rdd.flatMap(find_anomalies)
    return anomalies_rdd.sortBy(lambda x: x[3], ascending=False).take(10)

# Exécution
print("Détection des films aberrants (Z-score > 2)...")
resultat_ex9 = solve_exercice_9(imdb_rdd)

# Affichage
print(f"{'Film':<30} | {'Genre':<15} | {'Runtime':<10} | {'Z-score':<10}")
print("-" * 75)
for title, genre, rt, z in resultat_ex9:
    print(f"{title[:30]:<30} | {genre:<15} | {rt:<10} | {z:<10.2f}")

Détection des films aberrants (Z-score > 2)...
Film                           | Genre           | Runtime    | Z-score   
---------------------------------------------------------------------------
The Angry Grandpa Show         | Drama           | 3692080.0  | 1182.44   
The Angry Grandpa Show         | Comedy          | 3692080.0  | 928.87    
Logistics                      | Documentary     | 51420.0    | 462.84    
The Angry Grandpa Show         | Reality-TV      | 3692080.0  | 456.42    
100                            | Animation       | 59460.0    | 435.76    
Unsual What if (Old Videos)    | Action          | 37440.0    | 418.60    
Ambiancé                       | Documentary     | 43200.0    | 388.79    
Unsual What if (Old Videos)    | Animation       | 37440.0    | 274.33    
Svalbard Minute by Minute      | Adventure       | 13319.0    | 270.24    
Carnets Filmés (Liste Complète | Documentary     | 28643.0    | 257.64    


### Interprétation de l'Exercice 9 : Détection des anomalies de runtime par genre (Z-score > 2)

L'objectif de cet exercice était d'identifier les films dont le runtime est statistiquement aberrant pour leur genre, en utilisant le Z-score. Un film est considéré comme une anomalie si son Z-score est supérieur à 2 (c'est-à-dire si sa durée est à plus de deux écarts-types de la moyenne de son genre).

**Résultats obtenus (Top 10 des anomalies) :**

| Film                           | Genre           | Runtime    | Z-score   |
| :----------------------------- | :-------------- | :--------- | :-------- |
| The Angry Grandpa Show         | Drama           | 3692080.0  | 1182.44   |
| The Angry Grandpa Show         | Comedy          | 3692080.0  | 928.87    |
| Logistics                      | Documentary     | 51420.0    | 462.84    |
| The Angry Grandpa Show         | Reality-TV      | 3692080.0  | 456.42    |
| 100                            | Animation       | 59460.0    | 435.76    |
| Unsual What if (Old Videos)    | Action          | 37440.0    | 418.60    |
| Ambiancé                       | Documentary     | 43200.0    | 388.79    |
| Unsual What if (Old Videos)    | Animation       | 37440.0    | 274.33    |
| Svalbard Minute by Minute      | Adventure       | 13319.0    | 270.24    |
| Carnets Filmés (Liste Complète | Documentary     | 28643.0    | 257.64    |

**Analyse des résultats :**

*   **Présence de runtimes extrêmes :** Les résultats confirment l'existence de titres avec des runtimes absolument gigantesques, déjà suspectés par l'écart-type très élevé de l'exercice 3 et l'histogramme de l'exercice 6.
    *   **"The Angry Grandpa Show"** apparaît comme l'anomalie la plus significative, avec un runtime de 3,692,080 minutes (soit plus de 6 ans de contenu en continu !). Ce titre est classé dans plusieurs genres (Drama, Comedy, Reality-TV) et génère un Z-score extrêmement élevé pour chacun d'eux, indiquant une déviation massive par rapport à la durée moyenne de ces genres. Il s'agit très probablement d'une compilation de tous les épisodes ou d'une erreur dans la base de données.
    *   **"Logistics"** et **"Ambiancé"** sont des films expérimentaux connus pour leurs durées extrêmes (respectivement 51420 minutes, soit ~35 jours, et 43200 minutes, soit ~30 jours). Leur classification en "Documentary" les rend fortement anormaux par rapport aux durées standards de ce genre.
*   **Z-score comme indicateur d'anomalie :** Le Z-score > 2 est un critère efficace pour détecter ces valeurs aberrantes. Les scores obtenus (allant jusqu'à 1182.44) sont bien au-delà de ce seuil, mettant en évidence des titres dont la durée est non seulement inhabituelle mais potentiellement erronée ou représente un cas très particulier (film expérimental, compilation exhaustive, etc.).
*   **Dépendance au genre :** L'approche par genre est cruciale. Un film de 100 minutes pourrait être une anomalie dans le genre "Short", mais pas du tout dans le genre "Movie". En calculant le Z-score par genre, nous identifions des anomalies *relatives* à leur contexte.
*   **Utilisation pratique :** Cette technique est extrêmement utile pour le nettoyage de données (identifier des erreurs), pour des analyses de niche (détecter des œuvres expérimentales) ou pour comprendre la distribution des données au-delà des moyennes agrégées.

En conclusion, l'exercice 9 démontre avec succès la capacité de MapReduce et des statistiques descriptives à identifier des anomalies significatives dans un grand dataset, révélant des informations précieuses sur la qualité et la nature des données.

# Exercice 10 : Join complexe (Basics & Ratings) et Score Combiné

### 1. Objectif
Calculer la valeur maximale du produit `(runtime * avg_rating)` pour chaque genre. Cela permet d'identifier, pour chaque catégorie, le film qui combine le mieux une longue durée et une excellente note.

### 2. Logique MapReduce
Le traitement s'effectue en trois étapes :
1. **Préparation du dataset Ratings :** Charger le fichier `title.ratings.tsv` et créer un RDD de type `(tconst, averageRating)`.
2. **Jointure (JOIN) :** Joindre le RDD de base (Basics) avec celui des notes en utilisant l'identifiant unique du film (`tconst`) comme clé de jointure.
3. **Agrégation par Genre :**
    * **Map :** Calculer le produit `runtime * rating` pour chaque genre associé au film.
    * **Reduce :** Utiliser `reduceByKey` avec la fonction `max` pour conserver uniquement la valeur la plus élevée par genre.

## 2. Implémentation Spark (RDD)
Pour cet exercice, nous devons d'abord télécharger et préparer le second fichier.

In [ ]:

# --- ÉTAPE PRÉALABLE : Téléchargement du fichier Ratings ---
!wget -q https://datasets.imdbws.com/title.ratings.tsv.gz
!gunzip -f title.ratings.tsv.gz

def solve_exercice_10(basics_rdd):
    # 1. Charger et préparer Ratings RDD
    raw_ratings = sc.textFile("title.ratings.tsv")
    header_r = raw_ratings.first()

    # RDD format: (tconst, rating)
    ratings_rdd = raw_ratings.filter(lambda x: x != header_r) \
                             .map(lambda x: x.split("\t")) \
                             .map(lambda x: (x[0], float(x[1])))

    # 2. Préparer Basics RDD pour la jointure
    # RDD format: (tconst, (genres_list, runtime))
    basics_for_join = basics_rdd.map(lambda x: (x[0], (x[8].split(','), x[7]))) \
                                .filter(lambda x: x[1][1] != '\\N') \
                                .mapValues(lambda x: (x[0], float(x[1])))

    # 3. Effectuer la jointure (JOIN)
    # Resultat: (tconst, ((genres, runtime), rating))
    joined_rdd = basics_for_join.join(ratings_rdd)

    # 4. Mapper vers (Genre, Produit)
    def map_to_product(data):
        tconst, ((genres, runtime), rating) = data
        product = runtime * rating
        return [(g.strip(), product) for g in genres if g != '\\N']

    # 5. Calculer le Max par genre
    result = joined_rdd.flatMap(map_to_product) \
                       .reduceByKey(lambda a, b: max(a, b)) \
                       .sortBy(lambda x: x[1], ascending=False) \
                       .collect()

    return result

# Exécution
print("Exécution de la jointure complexe et calcul du score max par genre...")
resultat_ex10 = solve_exercice_10(imdb_rdd)

# Affichage
print(f"{'Genre':<20} | {'Max(Runtime x Rating)':<20}")
print("-" * 45)
for genre, score in resultat_ex10:
    print(f"{genre:<20} | {score:<20.2f}")

Exécution de la jointure complexe et calcul du score max par genre...
Genre                | Max(Runtime x Rating)
---------------------------------------------
Drama                | 29167432.00         
Comedy               | 29167432.00         
Reality-TV           | 29167432.00         
Animation            | 451896.00           
Documentary          | 329088.00           
Sport                | 113101.80           
Talk-Show            | 113101.80           
Adventure            | 109215.80           
Adult                | 60139.20            
Mystery              | 32760.00            
History              | 31365.00            
Crime                | 30132.00            
Musical              | 25200.00            
Music                | 25200.00            
Horror               | 25063.50            
Family               | 24600.00            
Action               | 21645.00            
Game-Show            | 18691.60            
News                 | 18144.00            
Rom

### Interprétation de l'Exercice 10 : Join complexe (Basics & Ratings) et Score Combiné

L'objectif de cet exercice était de combiner les informations de `title.basics.tsv` (runtimes, genres) et `title.ratings.tsv` (notes moyennes) pour calculer, pour chaque genre, la valeur maximale du produit `(runtime * avg_rating)`. Ce score combiné cherche à identifier le film qui excelle le plus dans la combinaison d'une longue durée et d'une excellente note au sein de son genre.

**Résultats obtenus (Top des genres) :**

| Genre              | Max(Runtime x Rating) |
| :----------------- | :-------------------- |
| Drama              | 29167432.00           |
| Comedy             | 29167432.00           |
| Reality-TV         | 29167432.00           |
| Animation          | 451896.00             |
| Documentary        | 329088.00             |
| Sport              | 113101.80             |
| Talk-Show          | 113101.80             |
| Adventure          | 109215.80             |
| Adult              | 60139.20              |
| Mystery            | 32760.00              |
| History            | 31365.00              |
| Crime              | 30132.00              |
| Musical            | 25200.00              |
| Music              | 25200.00              |
| Horror             | 25063.50              |
| Family             | 24600.00              |
| Action             | 21645.00              |
| Game-Show          | 18691.60              |
| News               | 18144.00              |
| Romance            | 16000.00              |
| Fantasy            | 14400.00              |
| War                | 13960.60              |
| Biography          | 13380.00              |
| Sci-Fi             | 10979.40              |
| Short              | 9936.00               |
| Thriller           | 7220.00               |
| Western            | 2763.90               |
| Film-Noir          | 991.60                |

**Analyse des résultats :**

*   **Dominance des "Hyper-runtimes" :** Les genres "Drama", "Comedy", et "Reality-TV" affichent un score combiné maximal de 29,167,432.00. Ces scores sont astronomiques et proviennent très probablement du même titre ou des mêmes titres "anormaux" avec des runtimes de plusieurs millions de minutes identifiés dans l'exercice 9 (e.g., "The Angry Grandpa Show"). Si ce titre a une note moyenne respectable (même 8/10 par exemple), le produit de son runtime colossal par cette note donnera un score combiné extrêmement élevé, écrasant tous les autres films de durées conventionnelles.
*   **Impact des anomalies :** Cet exercice, tout comme l'exercice 5, met en lumière l'influence des valeurs aberrantes de runtime. Le score `(runtime * avg_rating)` favorise les films à très longue durée, et si ces films ont aussi de bonnes notes, ils deviennent des "outliers" massifs dans ce classement.
*   **Scores "réalistes" :** En dessous de ces trois genres "extrêmes", les scores diminuent drastiquement vers des valeurs plus "réalistes". Par exemple, "Animation" a un score maximal de 451,896.00, "Documentary" 329,088.00, etc. Ces scores sont encore élevés et reflètent la possibilité de très longues séries d'animation ou de documentaires avec de très bonnes notes. Pour les genres comme "Short", "Thriller" ou "Western", les scores sont bien inférieurs, ce qui est attendu car ces genres sont moins susceptibles d'avoir des titres avec des durées extrêmes combinées à des notes élevées.
*   **Utilité du calcul :** Malgré la domination des anomalies, ce calcul est pertinent pour des objectifs spécifiques. Par exemple, si l'on souhaite identifier les contenus qui ont réussi à maintenir une haute qualité (bonne note) sur une très longue durée, ce score serait un bon indicateur (après avoir filtré ou compris les anomalies extrêmes).

Cet exercice démontre la capacité de Spark RDD à effectuer des jointures complexes entre de grands datasets et à réaliser des agrégations basées sur des fonctions personnalisées. Il illustre également l'importance de la détection et de la compréhension des anomalies dans les données, car elles peuvent fortement biaiser les analyses agrégées.